In [1]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
#!pip install ipywidgets

import re
import unidecode
from spellchecker import SpellChecker
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import datadetox as detox
import importlib

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [169]:
pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 7):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

#fitragem série B comjunto 5
df_list[4] = df_list[4][df_list[4]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)

DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Convertendo padrão com '/'
DataFrame 2: Convertendo padrão com '/'
DataFrame 3: Convertendo padrão com '/'
DataFrame 4: Convertendo padrão ISO com '-'
DataFrame 5: Já é datetime64, mantido como está.


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [170]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [171]:
pd.set_option('display.max_rows', None)
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

df.sort_values(by=['Mandante', 'Visitante','edition','Data'], ascending=[True, True, True, True], inplace=True)
#df.head(10)

In [174]:
importlib.reload(detox)

# Agrupar e aplicar a função
df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)

# Aplica o agrupamento e mescla as linhas sem conflito
df_mesclado = (
    df_ajustado.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado = df_mesclado[df_mesclado['edition']<2024]

df_group = df_mesclado.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(
    lambda group: detox.ajustar_datas_semdiff(group,keep=True))

df_mesclado2 = (
    df_group.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado2.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx',index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_13184\2427877954.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)
C:\Users\USER\AppData\Local\Temp\ipykernel_13184\2427877954.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(detox.mesclar_linhas_sem_conflito)
C:\Users\USER\AppData\Local\Temp\ipykernel_13184\2427877954.py:9: DeprecationWarn

Garantindo partidas por temporada

In [272]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
importlib.reload(detox)

df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx')

#ajustando sao paulo e sao caetano jogo interrompido
df.drop(1007, inplace=True)

# ajustando jogo do flamengo
df.loc[2835, 'Data'] = pd.to_datetime('2009-11-07')
df.drop(2834, inplace=True)

# ajustando jogo do BOTAFOGO
df.loc[1969, 'Data'] = pd.to_datetime('2009-07-29')
df.drop(1968, inplace=True)

# ajustando jogo da chapecoense
df.drop(5753, inplace=True)

#2005
df.drop([1137,1137,1207,1231,1257,1284,1307,1310,1328,1341,1375,1379], inplace=True)
df.loc[1138, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1208, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1232, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1258, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1285, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1308, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1311, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1329, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1342, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1376, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1380, 'Data'] = pd.to_datetime('2009-10-19')

#2007 
df.drop([1970,1971], inplace=True)


detox.verifica_partidas_edi(df,'edition','Mandante')
detox.verifica_partidas_edi(df,'edition','Visitante')

df.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx',index=False)

Edições com número desigual de partidas por time: edition
2016    2
Name: n_partidas, dtype: int64
Edições com número desigual de partidas por time: edition
2016    2
Name: n_partidas, dtype: int64


Meslclando Colunas

In [8]:
df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx')
df.head(10)
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405 entries, 0 to 8404
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Temporada           8405 non-null   int64         
 1   Data                8405 non-null   datetime64[ns]
 2   Mandante            8405 non-null   object        
 3   Visitante           8405 non-null   object        
 4   arbitro             6459 non-null   object        
 5   publico             6459 non-null   float64       
 6   publico_max         3906 non-null   float64       
 7   gols_mandante       7788 non-null   float64       
 8   mandante_Placar     8404 non-null   float64       
 9   Mandante Placar     7261 non-null   float64       
 10  mandante_Placar_3   8024 non-null   float64       
 11  Gol Mandante        3799 non-null   float64       
 12  placar_mandante     4788 non-null   float64       
 13  gols_visitante      7788 non-null   float64     

In [16]:
pd.set_option('display.max_columns', None)
gols_mandante = ["gols_mandante","mandante_Placar","Mandante Placar","mandante_Placar_3","Gol Mandante","placar_mandante"]
gols_visitante = ["gols_visitante","visitante_Placar","Visitante Placar","visitante_Placar_3","Gol Visitante","placar_visitante"]

df[gols_mandante].nunique(axis=1,dropna=True).value_counts()
df[gols_visitante].nunique(axis=1,dropna=True).value_counts()

df['gols_mandante_moda'] = df[gols_mandante].mode(axis=1).iloc[:, 0]
df['gols_visitante_moda'] = df[gols_visitante].mode(axis=1).iloc[:, 0]
df.drop(columns=gols_mandante+gols_visitante, inplace=True)


In [18]:
colunas_hora = ["Horário", "hora", "hora_3", "Hora", "hora_partida"]
df[colunas_hora].nunique(axis=1,dropna=True).value_counts()

2    5672
1    2176
3     556
0       1
Name: count, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405 entries, 0 to 8404
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Temporada            8405 non-null   int64         
 1   Data                 8405 non-null   datetime64[ns]
 2   Mandante             8405 non-null   object        
 3   Visitante            8405 non-null   object        
 4   arbitro              6459 non-null   object        
 5   publico              6459 non-null   float64       
 6   publico_max          3906 non-null   float64       
 7   arena                8404 non-null   object        
 8   Arena                7261 non-null   object        
 9   arena_3              8024 non-null   object        
 10  Estádio              3799 non-null   object        
 11  estadio              1944 non-null   object        
 12  mandante_Estado      8404 non-null   object        
 13  Horário              7250 non-nul